<a href="https://colab.research.google.com/github/onun1059/Resnet-18/blob/main/%EC%A2%85%EC%84%A4%EA%B8%B0%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN 모델에서 튜닝 가능한 하이퍼파라미터
* Convolutional layers: 필터의 갯수, 필터의 크기, stride값, zero-padding의 유무
* Pooling layers: Pooling방식 선택(MaxPool or AvgPool), Pool의 크기, Pool stride 값(overlapping)
* Fully-connected layers: 넓이(width)
* 활성함수(Activation function): ReLU(가장 주로 사용되는 함수), SoftMax(multi class classification), Sigmoid(binary classification)
* Loss function: Cross-entropy for classification, L1 or L2 for regression
* 최적화(Optimization) 알고리즘과 이것에 대한 hyperparameter(보통 learning rate): SGD(Stochastic gradient descent), SGD with momentum, AdaGrad, RMSprop
* Random initialization: Gaussian or uniform, Scaling
* https://halfundecided.medium.com/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-cnn-convolutional-neural-networks-%EC%89%BD%EA%B2%8C-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0-836869f88375

In [ ]:
# 1. 구글드라이브와 colab 연동
from google.colab import drive  # 구글드라이브와 연동하기 위한 라이브러리
drive.mount('/content/gdrive')

In [ ]:
# 2. 라이브러리 import
'''
pytorch, torchvision, matplotlib, numpy, tqdm, time 라이브러리가 요구된다.
'''
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

In [ ]:
# 3. 하이퍼 파라미터 설정
EPOCH = 5
batch_size = 8
learning_rate = 1e-3
momentum = 0.9

In [ ]:
# 4. torch에서 사용할 gpu 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

In [ ]:
# 5. Classification할 Class list 지정
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# 6. 이미지 전처리
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
    )

In [ ]:
# 7. (수정불필요) Dataset 저장 및 Tensor 형태로 변환
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [ ]:
# 8. Baseline CNN Network
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()  ######zero padding -> 가장자리 손실 사라진다, 정보가 중복되는 overfitting 방지
    self.conv1 = nn.Conv2d(3, 6, 5)   ##(input_channel, output_channel, kerner_size, stride=1), filter 5여서 32x32->28x28(padding 안 한 상태)
    self.pool1 = nn.MaxPool2d(2,2)    ##(kerner = filter, stride = filter 움직임 정도 , stride bigger-> image smaller)
    self.conv2 = nn.Conv2d(6, 16, 5)    ##pooling: correlation이 낮은 부분 삭제-> 결과값 크기 감소 (Max or average)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 10) # 클래스 개수 10

  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))     ## convolution layer = convolution 처리 + Activiation function(relu)
    x = self.pool2(F.relu(self.conv2(x)))     ## relu : 선형 함수 convlution에 비선형성 추가

    x = x.view(-1, 16 * 5 * 5) # -1 : batchsize
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))

    return x

In [ ]:
import torch.nn as nn
import torch.nn.functional as Fclass Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # max-pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

        # dropout layer
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # convolutional layers
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)

        # flatten and fully connected layers
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [ ]:
# 9. Network 선언 및 device 위에 올리기
net = Net().to(device)

In [ ]:
# 10. (수정 가능) Loss function으로 사용할 criterion 변수 생성
criterion = nn.CrossEntropyLoss()       ## 분류 문제여서 손실함수 크로스 엔트로피 사용

In [ ]:
# 11. Optimizer
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
# 12. Training
#수정# loss_ = []
n = len(trainloader)

for epoch in range(EPOCH):

  running_loss = 0.0
  start = time.time()

  #수정# for i, data in tqdm(enumerate(trainloader, 0)):
  for data in tqdm(trainloader):

    inputs, labels = data[0].to(device), data[1].to(device) # 배치 데이터
    optimizer.zero_grad() # 배치마다 optimizer 초기화
    outputs = net(inputs)
    loss = criterion(outputs, labels) # 크로스 엔트로피 손실함수 계산

    loss.backward() # backpropagation
    optimizer.step() # 가중치 최적화

    running_loss += loss.item()
  # !!주의!! weight 저장경로는 상황에 맞게 수정
  torch.save(net.state_dict() ,'/usr/local/lib/python3.10/dist-packages/google/colab/drive.py')

  #수정# loss_.append(running_loss / n)
  #수정# print('[%d] loss: %.3f' %(epoch + 1, running_loss / len(trainloader)))
  print('[%d] loss: %.3f' %(epoch + 1, running_loss / n))
  print("epoch time :", time.time()-start)


In [ ]:
#Evaluation
net = Net()
net.to(device)
# !!주의!! weight 저장경로는 상황에 맞게 수정
net.load_state_dict(torch.load('/usr/local/lib/python3.10/dist-packages/google/colab/drive.py'))

# (수정불가) Evaluation
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}  #dictionary 생성
total_pred = {classname: 0 for classname in classes}    #dictionary 생성

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad(): # autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높일 수 있다.
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) # labels.size = batch_size
        correct += (predicted == labels).sum().item()
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

In [ ]:
#Inference
def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(20,10))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(testloader)
#수정# images, labels = dataiter.next()
images, labels = next(dataiter)

# GroundTruth display
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(8)))

# Predicted display
net = Net()
net.to(device)
# !!주의!! weight 저장경로는 상황에 맞게 수정
net.load_state_dict(torch.load('/usr/local/lib/python3.10/dist-packages/google/colab/drive.py'))

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(8)))